#Unzip dataset from Drive

In [ ]:
!rm -rf "/content/sample_data"
!rm -rf "/content/myFinalDataset"
!rm -rd "/content/myDataset"
!unzip -qq "/content/drive/MyDrive/tirocinioWorkingDirectory/datasets/grid/finalDataset/myFinalDatasetCroppedNewEdition.zip" -d "/content/"

I'm stupid so let's move the files in the correct directory

In [ ]:
!mv "/content/content/myDataset/" "/content/myDataset"
!rm -rf "/content/content"

#Video generator

In [ ]:
!pip install keras-video-generators
#the following line is mandatory to solve a bug in the current release of keras-video-generators library
!sed -i '18s/.*/from tensorflow.keras.utils import Sequence/' "/usr/local/lib/python3.7/dist-packages/keras_video/generator.py"

In [1]:
import os, glob, tensorflow, keras_video.utils
from keras_video import VideoFrameGenerator
from tensorflow import keras

# use sub directories names as classes
datasetPath = os.path.join('..', 'contentOld', 'myDatasetExperimental')
classes = [i.split(os.path.sep)[3] for i in glob.glob(os.path.join(datasetPath, '*'))]
classes.sort()

# some global params
SIZE = (80, 40)
CHANNELS = 1
NBFRAME = 5
BS =  32

# pattern to get videos and classes
#glob_pattern='/content/myDataset/{classname}/*'
#glob_pattern_train = os.path.join(datasetPath, 'train', '{classname}', '*')
#glob_pattern_val = os.path.join(datasetPath, 'val', '{classname}', '*')
glob_pattern_train = os.path.join(datasetPath, '{classname}', '*')

# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=False,
    rotation_range=3,
    width_shift_range=.2,
    height_shift_range=.2)

# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern_train,
    nb_frames=NBFRAME, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False,
    split_val=0.3)


valid = train.get_validation_generator()

class again, validation count: 2466, train count: 5755
class at, validation count: 2428, train count: 5667
class bin, validation count: 2454, train count: 5727
class blue, validation count: 2467, train count: 5759
class by, validation count: 2465, train count: 5754
class lay, validation count: 2514, train count: 5866
class now, validation count: 2466, train count: 5757
class place, validation count: 2475, train count: 5778
class please, validation count: 2468, train count: 5759
class red, validation count: 2466, train count: 5757
class set, validation count: 2424, train count: 5658
class soon, validation count: 2466, train count: 5757
class white, validation count: 2466, train count: 5754
class with, validation count: 2465, train count: 5753
Total data: 14 classes for 80501 files for train
Total data: 14 classes for 34490 files for validation


# Goodies (will be useful)

In [2]:
import gc
from tensorflow.keras.layers import AveragePooling3D, AveragePooling2D, Conv2D, Conv3D, Dropout, MaxPooling2D, MaxPooling3D, BatchNormalization, TimeDistributed, LSTM, Dense, Flatten, GlobalMaxPool2D

#useful vars
EPOCHS = 100
INPUTSHAPE = (NBFRAME,) + SIZE + (CHANNELS,)
NUMBEROFCLASSES = len(classes)

class ClearMemory(tensorflow.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        tensorflow.keras.backend.clear_session()

# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
#path = os.path.join('.', 'content', 'chkp')
#path = 'C:\\Users\\vrlab\\Desktop\\ANTONIO_CACCIAPUOTI\\content\\chkp'
path = 'C:\\Users\\vrlab\\Desktop\\ANTONIO_CACCIAPUOTI\\contentOld\\chkp\\useless'
if not os.path.exists(path):
  os.mkdir(path)

callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1, patience=3, min_lr=0.0001, monitor="val_acc", factor=0.8, min_delta=0.003),
    keras.callbacks.ModelCheckpoint(
        os.path.join(path, 'weights.{epoch:02d}.hdf5'),
        verbose=1),
    keras.callbacks.EarlyStopping(patience=10, monitor="val_acc", min_delta=0.001),
    ClearMemory()
]

opt = keras.optimizers.Adam(0.01) #0.001

# Model creation and training (ConvLSTM2D)

In [3]:
import gc
import tensorflow as tf
from tensorflow.keras.layers import ConvLSTM2D, Dropout, Flatten, Dense, SimpleRNN, MaxPooling3D

def makeConvLSTM2D():
  model = tf.keras.Sequential()
  
  model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True, data_format = "channels_last", input_shape = (NBFRAME, SIZE[1], SIZE[0], CHANNELS)))
  model.add(Dropout(0.2))
  #model.add(AveragePooling3D()) #reduce dimension
  #model.add(BatchNormalization()) #standardize inputs
    
  model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = False))
  model.add(Dropout(0.2))
  #model.add(AveragePooling2D())
  #model.add(BatchNormalization())
    
  model.add(Flatten())
  
  #model.add(Dense(512, activation="relu"))
  #model.add(Dropout(0.3))

  model.add(Dense(256, activation="relu"))
  model.add(Dropout(0.2))

  model.add(Dense(128, activation="relu"))
  model.add(Dropout(0.2))
  
  model.add(Dense(NUMBEROFCLASSES, activation = "softmax"))

  return model


# Model creation and training (TimeDistributed CNN + LSTM)

In [3]:
def makeCNN():
  model = keras.Sequential()
  
  model.add(Conv2D(filters = 64, kernel_size = (3,3), input_shape = (SIZE[1], SIZE[0], CHANNELS), activation = 'relu'))
  #model.add(BatchNormalization()) #standardize inputs
  #model.add(MaxPooling2D()) #reduce dimension
  model.add(Dropout(0.2))
    
  model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
  #model.add(BatchNormalization())
  #model.add(MaxPooling2D())
  model.add(Dropout(0.2))
    
  model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
  #model.add(BatchNormalization())
  #model.add(MaxPooling2D())
  model.add(Dropout(0.2))
    
  model.add(Flatten())

  return model

def makeRNN():
  #make cnn first
  cnn = makeCNN()
  
  #create model
  model = keras.Sequential()
  
  #add cnn
  #model.add(TimeDistributed(cnn, input_shape=INPUTSHAPE))
  model.add(TimeDistributed(cnn, input_shape = (NBFRAME, SIZE[1], SIZE[0], CHANNELS)))

  #add rnn
  model.add(LSTM(64))
  
  #lets decide
  #model.add(Dense(1024, activation='relu'))
  #model.add(Dropout(.2))
  
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  
  #model.add(Dense(128, activation='relu'))
  #model.add(Dropout(.2))
  
  model.add(Dense(NUMBEROFCLASSES, activation='softmax'))

  return model
  




# Model creation (3D CNN)

In [6]:
def make3DCNN():
  model = keras.Sequential()  
  
  model.add(Conv3D(64, kernel_size=(1, 4, 4), padding='same', strides=(1,1,1), data_format = "channels_last", input_shape = (SIZE[1], SIZE[0], NBFRAME, CHANNELS)))
  model.add(MaxPooling3D(pool_size=(1, 4, 4), padding='same', strides=(1,1,1)))
  #model.add(BatchNormalization(center=True, scale=True))
  model.add(Dropout(0.3))

  model.add(Conv3D(64, kernel_size=(1, 4, 4), padding='same', strides=(1,1,1)))
  model.add(MaxPooling3D(pool_size=(1, 4, 4), padding='same', strides=(1,1,1)))
  #model.add(BatchNormalization(center=True, scale=True))
  model.add(Dropout(0.2))
  
  model.add(Flatten(data_format='channels_last'))
  
  #model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
  #model.add(Dropout(0.2))
  
  model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.2))
  
  model.add(Dense(NUMBEROFCLASSES, activation='softmax'))


  return model

# Training

In [4]:
## lets fit
    
model = makeRNN()
model.summary()
model.compile(opt, 'categorical_crossentropy', metrics=['acc'], run_eagerly=True)
history = model.fit(train, validation_data=valid, epochs=EPOCHS, callbacks=callbacks)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 5, 161024)         74496     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                41238784  
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 14)                3598      
Total params: 41,333,518
Trainable params: 41,333,518
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
2515/2515 [==============================] - ETA: 0s - loss: 2.6352 - acc: 0.0725
Epoch 00001: s

In [4]:
import pickle
with open('C:\\Users\\vrlab\\Desktop\\ANTONIO_CACCIAPUOTI\\contentOld\\chkp\\2910ConvLSTM2D\\history.pickle', 'wb') as handle:
    pickle.dump(history.history, handle)